In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget

## Parameters

In [10]:
dt = 0.01
r = 10.0
g = 9.81
H = np.eye(2)      # observation matrix
B = np.eye(2)      # control matrix
Q = np.eye(2)*0.00001 # covariance of process noise
R = np.eye(2)*0.1 # covariance of measurement noise

In [8]:
def compute_k_prime(H,P,R):
    k = P @ H.T @ np.linalg.inv(H @ P @ H.T + R)
    return k
    
def build_A():
    A = np.array([(1, dt), (-g*dt/r, .999)])
    
    return A

def generate_noise(covariance):
    sample = np.zeros((covariance.shape[0],1))
    
    for r in range(covariance.shape[0]):
        sample[r,0] = np.random.normal(0, np.sqrt(covariance[r,r]))
    
    return sample
            

def predict(x,A,u,B,P):
    x_new = A @ x + B @ u
    P_new = A @ P @ A.T + Q
    return x_new, P_new

def update(x,z,H,P,R):
    K = compute_k_prime(H, P, R)
    x_new = x + K @ (z - H @ x)
    P_new = P - K @ H @ P
    return x_new, P_new

## Create Simulaltion Data

In [4]:
n = 10000
x_init = np.array([0.3,0.5])
x = x_init
A = build_A()
x_history = np.zeros((2,n))
u_history = np.zeros((2,n))
t_history = np.zeros((1,n))
z_history = np.zeros((2,n))

z = np.zeros((2,n))


for i in range(n):
    u = np.zeros((2,))
    
    x = A @ x + B @ u
    
    u_history[0:2,i] = u
    x_history[0:2,i] = x
    t_history[0,i] = dt*i
    
    z_history[0:2,i] = H @ x + generate_noise(R).reshape(2,)


plt.plot(t_history[0,:], z_history[0,:])
plt.plot(t_history[0,:], x_history[0,:])
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Filter

In [12]:
n = 10000

P = np.eye(2)*0.01 # filter covariance

x_hat = np.zeros((2,))

x_hat_history = np.zeros((2,n))

for i in range(n):
    u = u_history[0:2,i]
    z = z_history[0:2,i]
    x_hat, P = predict(x_hat, A, u, B, P)
     
    x_hat, P = update(x_hat, z, H, P, R)
    
    x_hat_history[0:2,i] = x_hat
    
plt.figure()
plt.plot(t_history[0,:], z_history[0,:], 'r')
plt.plot(t_history[0,:], x_hat_history[0,:], 'b')
plt.plot(t_history[0,:], x_history[0,:], 'g')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …